In [4]:
import numpy as np
import scipy
import implicit
import pandas as pd
import os

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


In [36]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [37]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [38]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [39]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [40]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
# 유저 수
ratings['user_id'].nunique()

6039

In [42]:
# 영화 수
movies['movie_id'].nunique()

3883

In [43]:
# 데이터 합치기
data = pd.merge(movies, ratings)
data = data[["user_id", "title", "counts"]]

In [44]:
data

,user_id,title,counts
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
...,...,...,...
836473,5682,"Contender, The (2000)",3
836474,5812,"Contender, The (2000)",4
836475,5831,"Contender, The (2000)",3
836476,5837,"Contender, The (2000)",4


In [45]:
# 인기 많은 영화
title_count = data.groupby('title')['user_id'].count()
title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [46]:
# 유저별 몇 편의 영화를 시청했는지에 대한 통계
user_count = data.groupby("user_id")["title"].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [47]:
my_favorite = ["Terminator 2: Judgment Day (1991)", "Matrix, The (1999)", "Terminator, The (1984)",
              "Star Wars: Episode VI - Return of the Jedi (1983)", "Star Wars: Episode IV - A New Hope (1977)"]
favorite_genre = ["Action|Sci-Fi|Thriller", "Action|Sci-Fi|Thriller", "Action|Sci-Fi|Thriller",
                  "Action|Adventure|Romance|Sci-Fi|War", "Action|Adventure|Fantasy|Sci-Fi"]
my_watchlist = pd.DataFrame({"user_id": ["Kai"] * 5, "title": my_favorite, "counts": [5] * 5})

if not data.isin({'user_id':["Kai"]})['user_id'].any():
    data = data.append(my_watchlist) 

data.tail(10)

,user_id,title,counts
836473,5682,"Contender, The (2000)",3
836474,5812,"Contender, The (2000)",4
836475,5831,"Contender, The (2000)",3
836476,5837,"Contender, The (2000)",4
836477,5998,"Contender, The (2000)",4
0,Kai,Terminator 2: Judgment Day (1991),5
1,Kai,"Matrix, The (1999)",5
2,Kai,"Terminator, The (1984)",5
3,Kai,Star Wars: Episode VI - Return of the Jedi (1983),5
4,Kai,Star Wars: Episode IV - A New Hope (1977),5


In [87]:
movies.loc[movies.title.isin(my_favorite)]

,movie_id,title,genre
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1220,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [48]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, title indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [49]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,569,5
1,6039,2325,5
2,6039,1122,5
3,6039,1094,5


In [51]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['counts']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


In [53]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [54]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터

factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지<br/>
regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지<br/>
use_gpu : GPU를 사용할 것인지<br/>
iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [55]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [56]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

AlternatingLeastSquares 모델은 학습 진행 상황을 보고하는 내부 로그 출력을 가지고 있지 않기에 출력을 볼 수 있는 라이브러리인 tqdm 등을 사용하여 진행 상황을 확인

In [57]:
# 모델 훈련
from tqdm import tqdm
for epoch in tqdm(range(als_model.iterations)):
    als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [00:17<04:08, 17.72s/it]

  0%|          | 0/15 [00:00<?, ?it/s]

 13%|█▎        | 2/15 [00:35<03:50, 17.73s/it]

...

 87%|████████▋ | 13/15 [03:51<00:35, 17.89s/it]

  0%|          | 0/15 [00:00<?, ?it/s]

 93%|█████████▎| 14/15 [04:09<00:17, 17.88s/it]

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [04:27<00:00, 17.84s/it]


In [62]:
kai, terminator = user_to_idx['Kai'], title_to_idx['Terminator 2: Judgment Day (1991)']
kai_vector, terminator_vector = als_model.user_factors[kai], als_model.item_factors[terminator]

In [59]:
kai_vector

array([-0.26192775,  0.00324391,  0.16459623, -0.25128853, -0.11305313,
        0.0223823 ,  0.43803313, -0.04697328, -0.32233107,  0.19947007,
        0.18682246, -0.1958971 , -0.03966938,  0.2550013 , -0.30787423,
        0.0763041 ,  0.3757555 ,  0.03277843, -0.01294751, -0.13711983,
       -0.14332964, -0.37718943, -0.13577606,  0.05037242, -0.22525865,
        0.03887816,  0.09618823, -0.35605508, -0.17930576, -0.11464459,
       -0.01502715,  0.42936704, -0.23729008, -0.0300103 ,  0.02654161,
        0.26899612, -0.39870188,  0.27369207, -0.04249779, -0.3094252 ,
       -0.4448683 ,  0.29291594,  0.195863  , -0.23778272, -0.03752761,
        0.04331159, -0.3367845 ,  0.02053168,  0.13049111, -0.18783866,
       -0.0516262 , -0.26161286,  0.2702553 ,  0.05853696,  0.03697446,
       -0.36763263, -0.28464308,  0.0414348 ,  0.09769193,  0.18406919,
        0.07317994, -0.31809646,  0.23822449, -0.42013627, -0.03465516,
        0.03799707,  0.21158628,  0.19616343,  0.21001202,  0.01

In [63]:
terminator_vector

array([-0.04677508,  0.03185684,  0.05677308, -0.04694456, -0.0458148 ,
       -0.01167413,  0.06043152, -0.00785084, -0.09553052,  0.01068602,
        0.02564722,  0.03275185, -0.05457415,  0.03472705, -0.04755674,
        0.00940424, -0.02073374,  0.07640566, -0.03142159, -0.03086702,
       -0.02366308, -0.07849782,  0.03568509, -0.01097285, -0.03110099,
        0.0052309 ,  0.00089297, -0.06347261,  0.02763183, -0.06008124,
        0.0182719 ,  0.03853738,  0.03067857,  0.02768863, -0.06234892,
        0.00653748, -0.09362143, -0.04074608,  0.02495825, -0.07085647,
       -0.12103415,  0.07127979,  0.04360901, -0.04277949,  0.00947019,
       -0.00652818, -0.10385071,  0.00323388,  0.05054592, -0.01876924,
       -0.02706371, -0.03966902,  0.10296991,  0.0447762 ,  0.06221429,
       -0.06125304, -0.01822632,  0.02876478, -0.04492927, -0.00494775,
        0.01986307, -0.04567296,  0.0727693 , -0.00362318, -0.01818115,
       -0.01125395,  0.07953065,  0.03153232,  0.03922938,  0.00

In [64]:
# kai과 terminator를 내적하는 코드
np.dot(kai_vector, terminator_vector)

0.72842866

In [65]:
# 모델이 나의 Sixth Sense, The (1999)에 대한 선호도를 어떻게 예측
ss = title_to_idx['Sixth Sense, The (1999)']
ss_vector = als_model.item_factors[ss]
np.dot(kai_vector, ss_vector)

0.2791407

In [66]:
# 모델이 나의 Saving Private Ryan (1998)에 대한 선호도를 어떻게 예측
ss = title_to_idx['Saving Private Ryan (1998)']
ss_vector = als_model.item_factors[ss]
np.dot(kai_vector, ss_vector)

0.11562459

In [67]:
# 모델이 나의 SMen in Black (1997)에 대한 선호도를 어떻게 예측
ss = title_to_idx['Men in Black (1997)']
ss_vector = als_model.item_factors[ss]
np.dot(kai_vector, ss_vector)

0.3425215

0.5 밑으로 떨어지는 값을 출력하여 모델이 선호하지 않는다고 생각된다.

#### 비슷한 영화 찾기

In [72]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [73]:
get_similar_title('Fugitive, The (1993)')

['Fugitive, The (1993)',
 'Hunt for Red October, The (1990)',
 'In the Line of Fire (1993)',
 'Speed (1994)',
 'Enemy of the State (1998)',
 'Patriot Games (1992)',
 'Matrix, The (1999)',
 'Die Hard (1988)',
 'Die Hard 2 (1990)',
 'Terminator 2: Judgment Day (1991)']

In [75]:
user = user_to_idx['Kai']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(1080, 0.7653179),
 (1098, 0.46534634),
 (1084, 0.43877718),
 (2377, 0.42552763),
 (1082, 0.41175616),
 (462, 0.40504897),
 (2657, 0.39800364),
 (1419, 0.34252146),
 (2507, 0.27914068),
 (439, 0.2715166),
 (523, 0.2711273),
 (949, 0.27072468),
 (1152, 0.26225773),
 (3387, 0.22916073),
 (1173, 0.22354488),
 (3474, 0.21409708),
 (1448, 0.21386692),
 (1260, 0.1972141),
 (1003, 0.19686624),
 (1081, 0.19538495)]

In [85]:
recommend = [idx_to_title[i[0]] for i in title_recommended]
recommend

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Alien (1979)',
 'Aliens (1986)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Raiders of the Lost Ark (1981)',
 'Jurassic Park (1993)',
 'Total Recall (1990)',
 'Men in Black (1997)',
 'Sixth Sense, The (1999)',
 'Fugitive, The (1993)',
 'Blade Runner (1982)',
 'Die Hard (1988)',
 'Back to the Future (1985)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Indiana Jones and the Last Crusade (1989)',
 'X-Men (2000)',
 'Hunt for Red October, The (1990)',
 'Jaws (1975)',
 'E.T. the Extra-Terrestrial (1982)',
 'Princess Bride, The (1987)']

In [86]:
movies.loc[movies.title.isin(recommend)]

,movie_id,title,genre
453,457,"Fugitive, The (1993)",Action|Thriller
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
537,541,Blade Runner (1982),Film-Noir|Sci-Fi
1023,1036,Die Hard (1988),Action|Thriller
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1179,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


추천한 영화중 하나를 골라서 어떤 영화가 얼마나 추천에 기여하는지를 출력하였다.<br/>
영화의 장르가 거의 Action, Sci-Fi가 포함되는 장르들이었다. 내가 관심있는 영화들의 장르가 다 Action과 Sci-Fi가 포함된 장르였다.<br/>
장르로 따지면 괜찮게 추천해 주는듯 하다.

In [92]:
for i in title_recommended:
    print(f"{idx_to_title[i[0]]}: {i[1]:.3f}")

Star Wars: Episode V - The Empire Strikes Back (1980): 0.765
Alien (1979): 0.465
Aliens (1986): 0.439
Star Wars: Episode I - The Phantom Menace (1999): 0.426
Raiders of the Lost Ark (1981): 0.412
Jurassic Park (1993): 0.405
Total Recall (1990): 0.398
Men in Black (1997): 0.343
Sixth Sense, The (1999): 0.279
Fugitive, The (1993): 0.272
Blade Runner (1982): 0.271
Die Hard (1988): 0.271
Back to the Future (1985): 0.262
Mad Max 2 (a.k.a. The Road Warrior) (1981): 0.229
Indiana Jones and the Last Crusade (1989): 0.224
X-Men (2000): 0.214
Hunt for Red October, The (1990): 0.214
Jaws (1975): 0.197
E.T. the Extra-Terrestrial (1982): 0.197
Princess Bride, The (1987): 0.195


Star Wars: Episode V - The Empire Strikes Back (1980)와 Alien (1979)이 가장 높은 기여도를 나타내었다.

- iterations 값을 30으로 바꾸어 실험

In [93]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [94]:
from tqdm import tqdm
for epoch in tqdm(range(als_model.iterations)):
    als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:35<17:04, 35.32s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

  7%|▋         | 2/30 [01:11<16:47, 35.97s/it]

...

 93%|█████████▎| 28/30 [16:32<01:10, 35.10s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

 97%|█████████▋| 29/30 [17:06<00:34, 34.89s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [17:42<00:00, 35.41s/it]


In [95]:
# 모델이 나의 Sixth Sense, The (1999)에 대한 선호도를 어떻게 예측
ss = title_to_idx['Sixth Sense, The (1999)']
ss_vector = als_model.item_factors[ss]
np.dot(kai_vector, ss_vector)

0.13793823

In [96]:
# 모델이 나의 SMen in Black (1997)에 대한 선호도를 어떻게 예측
ss = title_to_idx['Men in Black (1997)']
ss_vector = als_model.item_factors[ss]
np.dot(kai_vector, ss_vector)

-0.13703911

In [97]:
user = user_to_idx['Kai']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
recommend = [idx_to_title[i[0]] for i in title_recommended]
movies.loc[movies.title.isin(recommend)]

,movie_id,title,genre
453,457,"Fugitive, The (1993)",Action|Thriller
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
537,541,Blade Runner (1982),Film-Noir|Sci-Fi
1023,1036,Die Hard (1988),Action|Thriller
1081,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1182,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller
1250,1270,Back to the Future (1985),Comedy|Sci-Fi


iterations이 15일때와 다른 점은 Princess Bride, The (1987)가 빠지고 Predator (1987)가 추천 되었다는 것인데 개인적으로는 더 적절한 추천이라고 생각된다.<br/>
그런데 왜 Men in Black (1997)선호도가 -가 나오는지가 의문이다.

# 회고

- **Matrix Factorization(MF)**을 이용하여 추천시스템을 만들어 보았다. 행렬을 사용자의 특성과 영화의 특성벡터로 분해하여 내적을 통해 선호도를 보는 모델이었다.
- 또한 낭비되는 메모리를 최소화 하기 위해 **CSR(Compressed Sparse Row) Matrix**을 사용하였다. 기존 matrix를 data, indices, indptr 로 행렬을 압축시켜 표현하는 것을 배웠다.
- 생각보다 잘 추천해 주는것 같았다.
- 지표에 관한 부분은 추가적인 공부가 필요할 듯 하다.